# Biased SVM

Biased SVMs are a variant of the standard SVM method that penalize misclassified positive and negative examples differently.

In [64]:
import numpy as np
import pandas as pd

### Load the dataset

In [65]:
from data import load_scar, load_sar, load_pg

train, valid, test, c = load_scar()

### Learn a biased SVM

A biased SVM is trained regarding unlabeled data as negative. A biased SVM penalizes misclassified positive and negative examples differently. The best parameter is found according to $F'_1$ on the validation set.

In [66]:
from sklearn.svm import SVC
from utils import f1_prime

train_xs, train_ys, train_ss, train_es = train
valid_xs, valid_ys, valid_ss, valid_es = valid

best_f1_prime = -1.0
best_clf = None
for class_weight_pos in np.arange(0.7, 1.0, 0.05):
    class_weight = {
        0: 1.0 - class_weight_pos,
        1: class_weight_pos,
    }

    clf = SVC(class_weight=class_weight, random_state=0, probability=True).fit(train_xs, train_ss)

    valid_ss_hat = clf.predict(valid_xs)

    f1_prime_ = f1_prime(valid_ss, valid_ss_hat)
    if f1_prime_ > best_f1_prime:
        print(f"The best classifier is updated: class weight is {class_weight}.")
        best_f1_prime = f1_prime_
        best_clf = clf

The best classifier is updated: class weight is {0: 0.30000000000000004, 1: 0.7}.
The best classifier is updated: class weight is {0: 0.04999999999999982, 1: 0.9500000000000002}.


### Predict

In [67]:
test_xs, test_ys, test_ss, test_es = test

test_ys_hat = best_clf.predict(test_xs)
test_ys_prob = best_clf.predict_proba(test_xs)[:, 1]

### Evaluate the performance

In [68]:
from sklearn.metrics import f1_score

f1_score(test_ys, test_ys_hat)

0.9910714285714286

### Visualize the result

In [69]:
from utils import plot_x_y, plot_x_y_proba

In [70]:
plot_x_y(test_xs, test_ys)

In [71]:
plot_x_y(test_xs, test_ys_hat)

In [72]:
plot_x_y_proba(test_xs, test_ys_prob)